In [1]:
import xarray as xr
import einops

In [2]:
import matplotlib.pyplot as plt
import numpy as np

In [3]:
from tqdm import tqdm

In [4]:
def _rolling_window(a, window):
    shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
    strides = a.strides + (a.strides[-1],)
    return np.lib.stride_tricks.as_strided(
        a, shape=shape, strides=strides, writeable=False
    )

In [5]:
ds_rad = xr.open_dataset('/pd/data/CML/data/reference/radklim_yw_1min/2018_2020_radklim_yw_1_min.nc', engine='netcdf4')
ds_cml = xr.open_dataset('/pd/data/CML/data/processed/proc2021.001/proc_hess_amt/proc_cnn_gapstandard_2020_05.nc', engine='netcdf4')
ds_rad = ds_rad.reindex(time=ds_cml.time.values,method='nearest',tolerance='10s')
ds_cml['radar_rainfall'] = ds_rad.rainfall_amount
ds_cml = ds_cml.isel(cml_id=np.arange(10)).load()
ds_cml['r_median_tl'] = ds_cml.txrx.rolling(time=60*72, min_periods=60*3).median()
ds_cml['tl_norm'] = ds_cml['txrx']-ds_cml['r_median_tl']
ds_cml = ds_cml.sel(time=slice('2020-05-10','2020-05-15'))
tl = einops.rearrange(ds_cml.tl_norm.values, 'c i t -> t i c')
rr = ds_cml.radar_rainfall.values


  0%|          | 0/10 [00:00<?, ?it/s]


NameError: name 'll' is not defined

In [8]:
l = []
r = []
for i in tqdm(range(tl.shape[1])):
    tll = []
    for j in range(tl.shape[2]):
        tll.append(_rolling_window(tl[:,i,j], 180))
    r.append(_rolling_window(rr[:,i], 180))
    l.append(tll)
l = einops.rearrange(np.array(l), 'i c b t -> (b i) t c')
r = einops.rearrange(np.array(r), 'i b t -> (b i) t')
ds = xr.Dataset()
ds['tl'] = ('sample_number','timestep','channel_id',), l
ds['radar'] = ('sample_number','timestep',), r
ds['tl_valid'] = ds.tl.isnull().sum(dim='timestep').sum(dim='channel_id')==0

100%|██████████| 10/10 [00:00<00:00, 37854.73it/s]


In [10]:
ds = ds.sel(sample_number=ds.sample_number[ds.tl_valid])

In [12]:
ds.nbytes/1e9

0.331200329

In [13]:
ds.to_netcdf('/bg/fast/aihydromet/cml_wet_dry_radklim/example_data.nc')